# check colData for dups

It looks like the col data training set shares some sample ids with both the val and test test. How ever when we check the sample ids in the count data set. we do not find any duplicates? This seems strange because the number sample ids in the count data set matchs the number of sample ids including the dups?


**<span style="color:red;background-color:yellow">see 'do all the count sample ids have col data?'</span>**  
turns out the number of sampleId's in colData matches the number of sampleIds in the countData. How ever there are 4154 duplicate sampleIds in colData and 4154 count data sample is do not have colData!

**TODO**
- test training, val, and test colData. Do they have the missing ids? Was there a bug when we concat them?
- fix colData

In [1]:
import ipynbname
from IPython.display import display
import matplotlib.pyplot as plt

import pandas as pd
# display all rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# https://joelmccune.com/pandas-dataframe-to-markdown/
from pandas.io.clipboards import to_clipboard

import pathlib as pl
import pprint as pp
import numpy as np
import os
import sys

/data/tmp/ipykernel_3097906/3026182786.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
dataRoot = "/private/groups/kimlab/GTEx_TCGA/groupbyGeneTrainingSets"
dataRoot = "/scratch/aedavids/tmp"
colDataPath = f"{dataRoot}/GTEx_TCGA_ColData.csv"
colDataDF = pd.read_csv(colDataPath)
print(f'colDataDF.shape : {colDataDF.shape}' )
colDataDF.head()

colDataDF.shape : (26337, 6)


,sample_id,participant_id,category,gender,age,dataSet
0,GTEX-1117F-0226-SM-5GZZ7,GTEX-1117F,Adipose_Subcutaneous,Female,66.0,GTEx
1,GTEX-1117F-0526-SM-5EGHJ,GTEX-1117F,Artery_Tibial,Female,66.0,GTEx
2,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F,Heart_Atrial_Appendage,Female,66.0,GTEx
3,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F,Breast_Mammary_Tissue,Female,66.0,GTEx
4,GTEX-1117F-3226-SM-5N9CT,GTEX-1117F,Brain_Cortex,Female,66.0,GTEx


In [3]:
print( len( colDataDF.loc[:, "sample_id"].unique() ))
print(f'!!!!!!!!!!!!! there are dups' )

22183
!!!!!!!!!!!!! there are dups


# check training sets

In [4]:
klDataRoot = "/private/groups/kimlab/GTEx_TCGA/groupbyGeneTrainingSets"

trainPath = f'{klDataRoot}/GTEx_TCGA_TrainColData.csv'
trainColDataDF = pd.read_csv(trainPath)
print(f'trainColDataDF.shape : {trainColDataDF.shape}')

valPath = f'{klDataRoot}/GTEx_TCGA_ValidateColData.csv'
valColDataDF = pd.read_csv(valPath)
print( f'\nvalColDataDF.shape : {valColDataDF.shape}' )

testPath = f'{klDataRoot}/GTEx_TCGA_TestColData.csv'
testColDataDF = pd.read_csv(testPath)
print(f'\ntestColDataDF.shape : {testColDataDF.shape}')

trainColDataDF.shape : (15801, 6)

valColDataDF.shape : (5268, 6)

testColDataDF.shape : (5268, 6)


In [5]:
def checkForDups(df, label="") :
    num = df.loc[:, "sample_id"].shape[0]
    numUnique = len( df.loc[:, "sample_id"].unique() )
    print(f'{label} num sample id: {num} numUnique : {numUnique}' )

print(f'!!!!!!! there are duplicated in the training col data set !!!!!!!!')
checkForDups(colDataDF, 'uber')
print()

checkForDups(trainColDataDF, 'train')
print()

checkForDups(valColDataDF, 'val')
print()

checkForDups(testColDataDF, 'test')

!!!!!!! there are duplicated in the training col data set !!!!!!!!
uber num sample id: 26337 numUnique : 22183

train num sample id: 15801 numUnique : 15801

val num sample id: 5268 numUnique : 5268

test num sample id: 5268 numUnique : 5268


In [6]:
print( 15801 + 5268 + 5268 )
print(f'number of dup sample ids {26337 - 22183}')

26337
number of dup sample ids 4154


In [7]:
print(f'check sampleIds interesections. The should be the empty set\n')
print(f'test and validate are disjoint as expected')
print(f'both test and validated share samples some sample with training data set\n')

trainSampleIdSet = set( trainColDataDF.loc[:, "sample_id"] )
valSampleIdSet   = set( valColDataDF.loc[:, "sample_id"] )
testSampleIdSet  = set( testColDataDF.loc[:, "sample_id"] )

print(f'len(trainSampleIdSet.intersection(valSampleIdSet) : {len(trainSampleIdSet.intersection(valSampleIdSet))}' )
print(f'len(trainSampleIdSet.intersection(testSampleIdSet) : {len(trainSampleIdSet.intersection(testSampleIdSet))}' )
print(f'len(valSampleIdSet.intersection(testSampleIdSet) : {len(valSampleIdSet.intersection(testSampleIdSet))}' )

check sampleIds interesections. The should be the empty set

test and validate are disjoint as expected
both test and validated share samples some sample with training data set

len(trainSampleIdSet.intersection(valSampleIdSet) : 2090
len(trainSampleIdSet.intersection(testSampleIdSet) : 2064
len(valSampleIdSet.intersection(testSampleIdSet) : 0


In [8]:
print(f'find duplicates in uber col data')
duplicateRows = colDataDF.duplicated()
print(f'duplicateRows[0:5] : {duplicateRows[0:5]}')
print(f'sum(duplicateRows) : {sum(duplicateRows)}')

find duplicates in uber col data
duplicateRows[0:5] : 0    False
1    False
2    False
3    False
4    False
dtype: bool
sum(duplicateRows) : 4154


In [9]:
colDataDuplicatedDF = colDataDF.loc[duplicateRows , :]
print(f'{colDataDuplicatedDF.shape}')
colDataDuplicatedDF.head()

(4154, 6)


,sample_id,participant_id,category,gender,age,dataSet
15802,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F,Heart_Atrial_Appendage,Female,66.0,GTEx
15804,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F,Breast_Mammary_Tissue,Female,66.0,GTEx
15807,GTEX-1117F-3226-SM-5N9CT,GTEX-1117F,Brain_Cortex,Female,66.0,GTEx
15808,GTEX-111CU-0326-SM-5GZXO,GTEX-111CU,Lung,Male,57.0,GTEx
15809,GTEX-111CU-0426-SM-5GZY1,GTEX-111CU,Spleen,Male,57.0,GTEx


In [10]:
colDataDuplicatedDF.groupby(['dataSet']).count()

,sample_id,participant_id,category,gender,age
dataSet,,,,,
GTEx,4154,4154,4154,4154,4154


In [11]:
colDataDuplicatedDF.groupby(['category']).count().head()

,sample_id,participant_id,gender,age,dataSet
category,,,,,
Adipose_Subcutaneous,149,149,149,149,149
Adipose_Visceral_Omentum,137,137,137,137,137
Adrenal_Gland,70,70,70,70,70
Artery_Aorta,98,98,98,98,98
Artery_Coronary,62,62,62,62,62


# Are there dups in the count data sets?

In [12]:
%%time
dataRoot = "/private/groups/kimlab/GTEx_TCGA/groupbyGeneTrainingSets"
dataRoot = "/scratch/aedavids/tmp"

# it is faster to read the data, then set the index.
# this probably has to do with the way the hdf5 file was saved to disk
normalizedGroupbyHDF5Path = f'{dataRoot}/GTEx_TCGA_NormalizedGroupby.hdf5'
normalizedGroupbyHDF5_DF = pd.read_hdf(normalizedGroupbyHDF5Path)
normalizedGroupbyHDF5_DF =  normalizedGroupbyHDF5_DF.set_index( 'geneId' )
print( f'normalizedGroupbyHDF5_DF.shape : {normalizedGroupbyHDF5_DF.shape}' )
normalizedGroupbyHDF5_DF.columns[0:3]
print()

normalizedGroupbyHDF5_DF.shape : (74777, 26337)

CPU times: user 16.2 s, sys: 18.6 s, total: 34.8 s
Wall time: 36 s


In [13]:
print(normalizedGroupbyHDF5_DF.columns[0:3])
print(normalizedGroupbyHDF5_DF.columns[-3:])
len( normalizedGroupbyHDF5_DF.columns.unique() )

Index(['GTEX-1117F-0226-SM-5GZZ7', 'GTEX-1117F-0526-SM-5EGHJ',
       'GTEX-1117F-0726-SM-5GIEN'],
      dtype='object')
Index(['UVM-WC-A87Y-TP', 'UVM-WC-AA9A-TP', 'UVM-YZ-A982-TP'], dtype='object')


26337

In [14]:
len( np.unique( normalizedGroupbyHDF5_DF.columns) )

26337

In [15]:
countDuplicatedSeries = normalizedGroupbyHDF5_DF.columns.duplicated()
sum( countDuplicatedSeries )

0

In [33]:
print(f'?????? looks like dups in colData but not count data, yet the number of sample match?')

?????? looks like dups in colData but not count data, yet the number of sample match?


In [17]:
dupSampleIds = colDataDuplicatedDF.loc[:, "sample_id"]
testDupIds = dupSampleIds[0:3].values
print(testDupIds)


print()
selectTestDupRows = colDataDF.loc[:, "sample_id"].isin(testDupIds)
colDataDF.loc[selectTestDupRows , :].sort_values(by="sample_id")

['GTEX-1117F-0726-SM-5GIEN' 'GTEX-1117F-2826-SM-5GZXL'
 'GTEX-1117F-3226-SM-5N9CT']



,sample_id,participant_id,category,gender,age,dataSet
2,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F,Heart_Atrial_Appendage,Female,66.0,GTEx
15802,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F,Heart_Atrial_Appendage,Female,66.0,GTEx
3,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F,Breast_Mammary_Tissue,Female,66.0,GTEx
15804,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F,Breast_Mammary_Tissue,Female,66.0,GTEx
4,GTEX-1117F-3226-SM-5N9CT,GTEX-1117F,Brain_Cortex,Female,66.0,GTEx
15807,GTEX-1117F-3226-SM-5N9CT,GTEX-1117F,Brain_Cortex,Female,66.0,GTEx


In [18]:
fooDF = normalizedGroupbyHDF5_DF.loc[:, testDupIds]
fooDF.iloc[0:5, :]

,GTEX-1117F-0726-SM-5GIEN,GTEX-1117F-2826-SM-5GZXL,GTEX-1117F-3226-SM-5N9CT
geneId,,,
(A)n,1.611646,10.992304,0.668533
(AAA)n,0.000000,0.000000,0.000000
(AAAAAAC)n,0.000000,0.000000,0.000000
(AAAAAAG)n,0.000000,0.000000,0.000000
(AAAAAAT)n,0.000000,0.000000,0.000000


# how did lump work?

In [19]:
# we combined counts using axis = 1

# Creating two sample DataFrames
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [7, 8, 9], 'C': [10, 11, 12]})

# Concatenating the DataFrames side by side
resultDF = pd.concat([df1, df2], axis=1)

display( resultDF )
print()
print(f'len(resultDF.columns) {len(resultDF.columns)}')
resultDF.columns.duplicated()

,A,B,A,C
0,1,4,7,10
1,2,5,8,11
2,3,6,9,12



len(resultDF.columns) 4


array([False, False,  True, False])

In [20]:
# we count  using  axis = 1

# Creating two sample DataFrames
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [7, 8, 9], 'C': [10, 11, 12]})
display(df1)
print()
display(df2)

# Concatenating the DataFrames side by side
print("\n***********")
resultDF = pd.concat([df1, df2], axis=1)

display( resultDF )
print()
print(f'len(resultDF.columns) {len(resultDF.columns)}')
resultDF.columns.duplicated()

,A,B
0,1,4
1,2,5
2,3,6


,A,C
0,7,10
1,8,11
2,9,12



***********


,A,B,A,C
0,1,4,7,10
1,2,5,8,11
2,3,6,9,12



len(resultDF.columns) 4


array([False, False,  True, False])

In [21]:
# we concat colData using default axis = 0


# Concatenating the DataFrames on top of each othger
print("\n***********")
resultDF = pd.concat([df1, df2], axis=0)

display( resultDF )
print()
print(f'len(resultDF.columns) {len(resultDF.columns)}')
resultDF.columns.duplicated()


***********


,A,B,C
0,1,4.0,NaN
1,2,5.0,NaN
2,3,6.0,NaN
0,7,NaN,10.0
1,8,NaN,11.0
2,9,NaN,12.0



len(resultDF.columns) 3


array([False, False, False])

# could there be white space in sample ids
No!

In [22]:
countCols = normalizedGroupbyHDF5_DF.columns
print(f'len(countCols) {len(countCols)} len(countCols.unique()) : {len(countCols.unique())}' )

stripedCountCols = list()
for c in countCols:
    stripedCountCols.append( c.strip() )

print(f'len(stripedCountCols) {len(stripedCountCols)}')
print(f'len(stripedCountCols.unique()) : {len(set(stripedCountCols))}' )

len(countCols) 26337 len(countCols.unique()) : 26337
len(stripedCountCols) 26337
len(stripedCountCols.unique()) : 26337


# Does dup sample id's change the filtered count data?

In [23]:
def aedwip():
    # Creating two sample DataFrames
    df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
    df2 = pd.DataFrame({'A': [1, 2, 3], 'C': [10, 11, 12]})
    display(df1)
    print()
    display(df2)
    
    # Concatenating the DataFrames side by side
    print("\n***********")
    resultDF = pd.concat([df1, df2], axis=1)
    display(resultDF)

    print("\n*****")
    df = resultDF.loc[:, "A"]
    display(df)

aedwip()

,A,B
0,1,4
1,2,5
2,3,6


,A,C
0,1,10
1,2,11
2,3,12



***********


,A,B,A,C
0,1,4,1,10
1,2,5,2,11
2,3,6,3,12



*****


,A,A
0,1,1
1,2,2
2,3,3


# do all the count sample ids have col data?
## **<span style="color:red;background-color:yellow">NO!</span>**

4154 count sample ids do not have col data!

In [31]:
def findMissingColDataSampleIds():
    '''
    make sure all the count data has colData
    '''
    countSampleIds = normalizedGroupbyHDF5_DF.columns
    print(f'len(countSampleIds) : {len(countSampleIds)}')
    
    colDataSampleIds = colDataDF.loc[:, "sample_id"]
    print(f'len(colDataSampleIds) : {len(colDataSampleIds)}')

    countSampleIdSet = set(countSampleIds)
    colDataSampleIdSet = set(colDataSampleIds)

    print()
    print( 'are their sample ids in the count data set that do not have colData?')
    print('YES!!!!')
    print(f'len( countSampleIdSet - colDataSampleIdSet) : {len( countSampleIdSet - colDataSampleIdSet)}' )

    print('\nare the col data sample ids that do not have cooresponding count data?')
    print('no')
    print(f'len(colDataSampleIdSet - countSampleIdSet) : {len( colDataSampleIdSet - countSampleIdSet)}' )

    missingSampleIds = countSampleIdSet - colDataSampleIdSet

    return missingSampleIds


missingColDataSampleIds = findMissingColDataSampleIds()

len(countSampleIds) : 26337
len(colDataSampleIds) : 26337

are their sample ids in the count data set that do not have colData?
YES!!!!
len( countSampleIdSet - colDataSampleIdSet) : 4154

are the col data sample ids that do not have cooresponding count data?
no
len(colDataSampleIdSet - countSampleIdSet) : 0


### Check was there a bug in concat? 
check to see if train, val, and test colData have missing sample ids
### **<span style="color:red;background-color:yellow">NO!</span>**

In [39]:
colDataSampleIdDict = {
    'colData_uber':  colDataDF.loc[:, "sample_id"],
    'colData_train': trainColDataDF.loc[:, "sample_id"],
    'colData_val':   valColDataDF.loc[:, "sample_id"],
    'colData_test':  testColDataDF.loc[:, "sample_id"],
}

for datasetName,sampleId in colDataSampleIdDict.items():
    print()
    print(f'{datasetName} {len(missingColDataSampleIds - set(sampleId))}' )
    print(f'{datasetName} {len( set(sampleId) - missingColDataSampleIds)}' )



colData_uber 4154
colData_uber 22183

colData_train 4154
colData_train 15801

colData_val 4154
colData_val 5268

colData_test 4154
colData_test 5268


# Do all the colData data sets have count data?